## Современные LLM

### План

1. Современные языковые модели. Статья: Aligning language models to follow instructions
1. Разница между pre-trained моделями и fine-tuned моделями на примере Llama 3.2-1B:
- Архитектура 
- Токенизаторы 
- Чат темплейт
3. Supervised finetuning
4. RLHF 


## Современные LLM

Все модели сейчас проходят три стадии обучения: pre-training, supervised fine-tuning и RLHF. Пройдемся по каждому из них: 

### Pre-training
Языковые модели учатся на задачу Next token prediction. Для этого обычно собирают огромные датасеты в автоматическом режиме. Стандартные источники данных: парсинг сайтов (с последующей очисткой от html-тегов и прочего) , распознование книг и так далее. Примеры датасетов: [c4](https://huggingface.co/datasets/allenai/c4), [RedPajama](https://github.com/togethercomputer/RedPajama-Data), [wikipedia](https://huggingface.co/datasets/wikimedia/wikipedia)

Для тестирования их способностей существует множество бенчмарков, например, [hellaswag](https://arxiv.org/abs/1905.07830), [OpenBookQA](https://huggingface.co/datasets/allenai/openbookqa), [WinoGrande](https://huggingface.co/datasets/allenai/winogrande). В таких бенчмарках чаще всего просят либо сгенерировать ответ в формате Multiple-Choice, либо сгенерировать продолжение. 

Такие модели обладают отличными знаниями мира засчет того, что видели много данных, но у них есть проблема: они не умеют отвечать в формате, полезном для человека. Посмотрим на примере Llama-3.2-1B

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

Попробуем попросить ее решить простую задачку

In [13]:
pipe_pretrained = pipeline("text-generation", model="unsloth/Llama-3.2-1B", max_new_tokens=512)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [5]:
task = "Найдите значение n в уравнении n + 2 = 6"

In [321]:
pipe_pretrained(task)

[{'generated_text': 'Найдите значение n в уравнении n + 2 = 6n + 1\nn = 1\nn = 2\nn = 3\nn = 4\nn = 5\nn = 6\nn = 7\nn = 8\nn = 9\nn = 10\nn = 11\nn = 12\nn = 13\nn = 14\nn = 15\nn = 16\nn = 17\nn = 18\nn = 19\nn = 20\nn = 21\nn = 22\nn = 23\nn = 24\nn = 25\nn = 26\nn = 27\nn = 28\nn = 29\nn = 30\nn = 31\nn = 32\nn = 33\nn = 34\nn = 35\nn = 36\nn = 37\nn = 38\nn = 39\nn = 40\nn = 41\nn = 42\nn = 43\nn = 44\nn = 45\nn = 46\nn = 47\nn = 48\nn = 49\nn = 50\nn = 51\nn = 52\nn = 53\nn = 54\nn = 55\nn = 56\nn = 57\nn = 58\nn = 59\nn = 60\nn = 61\nn = 62\nn = 63\nn = 64\nn = 65\nn = 66\nn = 67\nn = 68\nn = 69\nn = 70\nn = 71\nn = 72\nn = 73\nn = 74\nn = 75\nn = 76\nn = 77\nn = 78\nn = 79\nn = 80\nn = 81\nn = 82\nn = 83\nn = 84\nn = 85\nn = 86\nn = 87\nn = 88\nn = 89\nn = 90\nn = 91\nn = 92\nn = 93\nn = 94\nn = 95\nn = 96\nn = 97\nn = 98\nn = 99\nn = 100\nn = 101\nn ='}]

Что-то явно пошло не так. На самом деле, это хорошо решается, если сделать промпт в форме продолжения текста: 

In [322]:
task_fixed = "Найдите значение n в уравнении n + 2 = 6. Ответ: n = "

In [323]:
pipe(task_fixed)

[{'generated_text': 'Найдите значение n в уравнении n + 2 = 6. Ответ: n = 4.\nНайдите значение n в уравнении n + 2 = 6. Ответ: n = 4.\nНайдите значение n в уравнении n + 2 = 6. Ответ: n = 4.\nНайдите значение n в уравнении n + 2 = 6. Ответ: n = 4.\nНайдите значение n в уравнении n + 2 = 6. Ответ: n = 4.\nНайдите значение n в уравнении n + 2 = 6. Ответ: n = 4.\nНайдите значение n в уравнении n + 2 = 6. Ответ: n = 4.\nНайдите значение n в уравнении n + 2 = 6. Ответ: n = 4.\nНайдите значение n в уравнении n + 2 = 6. Ответ: n = 4.\nНайдите значение n в уравнении n + 2 = 6. Ответ: n = 4.\nНайдите значение n в уравнении n + 2 = 6. Ответ: n = 4.\nНайдите значение n в уравнении n + 2 = 6. Ответ: n = 4.\nНайдите значение n в уравнении n + 2 = 6. Ответ: n = 4.\nНайдите значение n в уравнении n + 2 = 6. Ответ: n = 4.\nНайдите значение n в уравнении n + 2 = 6. Ответ: n = 4.\nНайдите значение n в уравнении n + 2 = 6. Ответ: n = 4.\nНайдите значение n в уравнении n + 2 = 6. Ответ: n = 4.\nНайдите зн

Уже лучше, но это далеко от того, как себя ведет, например, ChatGPT. В чем причина? 

**Модели, которые предназначены для взаимодействия с пользователем проходят еще несколько стадий обучения. Обобщенно эти стадии называют Alignment (хотя сейчас все вкладывают в это понятие разные значения)**

Пошло это наименование со статьи [Aligning language models to follow instructions](https://openai.com/index/instruction-following/)

Посмотрим как с такими задачами справится чат-моделька:

In [8]:
pipe = pipeline("text-generation", model="unsloth/Llama-3.2-1B-Instruct", max_new_tokens=512)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Правда, нам придется немного поменять формат входных данных, так как чат модельки принимают данные в формате диалога. 

In [9]:
messages = [
    {"role": "user", "content": "Найдите значение n в уравнении n + 2 = 6"},
]

In [10]:
pipe(messages)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


[{'generated_text': [{'role': 'user',
    'content': 'Найдите значение n в уравнении n + 2 = 6'},
   {'role': 'assistant',
    'content': 'Чтобы найти значение n, мы можем вычесть 2 из обеих частей уравнения:\n\nn + 2 = 6\nn + 2 - 2 = 6 - 2\nn = 4\n\nИтак, значение n равно 4.'}]}]

Здесь мы видим такой ответ, к которому мы привыкли в ChatGPT. Самое интересное, что в самих моделей никаких отличий нет

In [14]:
pipe.model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [16]:
pipe_pretrained.model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer_pretrained = AutoTokenizer.from_pretrained("unsloth/Llama-3.2-1B")
model_pretrained = AutoModelForCausalLM.from_pretrained("unsloth/Llama-3.2-1B")

### Как достигается такая разница в ответах? 

## Supervised Fine-Tuning

Как чаще всего людей учат что-то делать? Дают какую-то задачу и показывают хорошее решение этой задачи. 

Здесь идея та же: давайте соберем набор данных в формате диалога пользователя и ассистента. Пользователь задает вопрос, ассистент отвеает ему. 

В идеале (как это делали в изначальной статье) ответы должны писать люди - эксперты в области. 

Таким образом, модель выучивает формат ответа и использует свои знания, полученные на предыдущей стадии, чтобы помогать пользователю.

Примеры таких датасетов: [UltraChat](https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k), [No-Robots](https://huggingface.co/datasets/HuggingFaceH4/no_robots), [MathInstruct](https://huggingface.co/datasets/TIGER-Lab/MathInstruct). 

Чтобы собирать такие датасеты крупные компании (OpenAI, Meta, Яндекс и другие) собирали целые отделы, состоящие из профессиональных редакторов, чтобы писать инструкции и ответы к ним. 

Далее, на полученных данных происходит стандартное обучение как и в предыдущей стадии (next-token prediction)

![SFT](https://habrastorage.org/webt/79/dr/np/79drnpe2spuzrttprryqkpjt8tk.jpeg)

### На самом деле, сейчас такие данные можно собирать и без дорогой разметки редакторами.

Есть несколько подходов, например, [Alpaca](https://crfm.stanford.edu/2023/03/13/alpaca.html), которые позволяют собирать данные и без них. 

Alpaca устроена следующим образом: 

- Выбирают 175 написанных людьми инструкций из датасета self-instruct
- Промптят GPT-3.5 (на тот момент она была SOTA) чтобы нагенерировать еще похожих инструкций
- Учатся на полученных данных


![Alpaca](https://crfm.stanford.edu/static/img/posts/2023-03-13-alpaca/alpaca_main.jpg)

Но есть и другие методы, например, [Instruction Pre-Training](https://arxiv.org/abs/2406.14491), [Mammoth](https://arxiv.org/abs/2405.03548)

Они сводятся к тому, что инструкции можно генерировать из тех же данных, на которых модель предобучается.

## Preference Learning (RLHF)

Но это еще не все. Несмотря на то, что SFT уже дает неплохое качество, настоящим прорывом стал метод RLHF, который впервые успешно применили OpenAI в [InstructGPT](https://openai.com/index/instruction-following/)
 
В этом методе предлагается повышать не только вероятности для "хороших" ответов, но и понижать вероятности для "плохих", например - токсичных, опасных или ложных. 

В чем суть подхода:

- На один запрос генерируется несколько вариантов ответа одной модели, например, с помощью изменения параметров генерации.
- Ответы ранжируются людьми: так, ответ без всех "изьянов", будет более предпочтительным, чем с ними
- Затем, на ответах обучается модель награды, которая выдает большую награду более "хорошему" ответу
- После этого используются методы Обучения с Подкреплением, а именно, PPO, чтобы максимизировать награду.

![RLHF](https://images.ctfassets.net/kftzwdyauwt9/12CHOYcRkqSuwzxRp46fZD/928a06fd1dae351a8edcf6c82fbda72e/Methods_Diagram_light_mode.jpg?w=3840&q=80&fm=webp)

Люди предпочитали ответы меньшей модели, обученной с RLHF, ответам бОльшей модели.

![img](https://gcdnb.pbrd.co/images/gi3riropQvQe.png?o=1)

# DPO


Но в то же время PPO это очень дорогой метод, который имеет много гиперпараметров. В его реализации очень легко допустить ошибки, которые тяжело раздебагать. 

В 2023 году вышла статья [Direct Preference Optimization](https://arxiv.org/abs/2305.18290)

Путем сложной матеши авторы вывели, что оптимальная политика выражается через саму функцию (поэтому статья называется Your Language Model is Secretly a Reward model), поэтому можно избежать проблем с PPO и с RL'ем в целом. 

![DPO](https://ghost.oxen.ai/content/images/2024/01/Screenshot-2024-01-26-at-3.49.01-PM.png)

Все эти (и другие) методы доступны в библиотеке [TRL](https://huggingface.co/docs/trl/en/index) 

- [SFT Trainer](https://huggingface.co/docs/trl/en/sft_trainer) 

- [Reward Modelling](https://huggingface.co/docs/trl/en/reward_trainer)

- [PPO Trainer](https://huggingface.co/docs/trl/en/ppo_trainer)

- [DPO Trainer](https://huggingface.co/docs/trl/en/dpo_trainer)

![TRL](https://gcdnb.pbrd.co/images/pQ7aghF2128t.png?o=1)

## Как оценить качество модели и понять, какая нужна именно вам?

## Существует множество бенчмарков, в которых проверяются определенные навыки моделей. 

Примеры бенчей: 

- [MT-Bench](https://arxiv.org/abs/2306.05685): на многоступенчатые диалоги (несколько переходов человек-ассистент) генерируются продолжения, а затем, оцениваются с помощью GPT4.
- [AlpacaEval](https://tatsu-lab.github.io/alpaca_eval/): сравнивают ответ модели с ответом ГПТ4, затем оценивают с помощью другой модели. Затем считают win-rate. 


- [ChatBot Arena](https://lmarena.ai): Пользователям дают возможность пообщаться с разными моделями (в том числе с платными) и задать им свои вопросы. Затем, дают пользователю выбрать тот ответ, что ему больше нравится из двух. На основе этого строят рейтинг моделей. 

- [LLM Arena](https://llmarena.ru): То же самое, но на русском языке.

Текущий лидерборд на арене

![Arena](https://gcdnb.pbrd.co/images/9FoqK20pMVoq.png?o=1)

## Какую модель выбрать?

### Почти всегда предпочтение стоит отдавать моделям, которые обучены отвечать в чат-формате. То есть, которые прошли alignment стадию. 

Учить свою модель с нуля почти всегда не выгодно финансово, а также нецелесообразно, если, конечно, вы не работаете в огромной компании, у которой есть ресурсы на это. 


Современные модели относительно "маленького" размера ([gemma2 9b](https://huggingface.co/google/gemma-2-9b-it), [llama-3.1 8B](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct)) неплохо справляются с общими задачами, на порядок лучше, чем "старые" модели даже больших размеров.

Большие модели ([gemma2 27b](https://huggingface.co/google/gemma-2-27b-it), [llama3.1 405b](https://huggingface.co/meta-llama/Llama-3.1-405B-Instruct)) зачастую даже не нужно дополнительно обучать под конкретные задачи (например, суммаризация или классификация), достаточно грамотно написать задачу модели и описать какой формат выходных данных от нее ожидается. 

Подробнее про prompt-engineering можно почитать [тут](https://platform.openai.com/docs/guides/prompt-engineering)